In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
import os

# 1. DEFINE THE MODEL ARCHITECTURE
# You MUST include this class definition exactly as it was during training.
# PyTorch saves the weights, but not the structure of the model.
class FSMTRegressor(nn.Module):
    def __init__(self, n_classes):
        super(FSMTRegressor, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        pooled_output = output.pooler_output
        output = self.drop(pooled_output)
        return self.out(output)

In [7]:
# Update this path if your folder is named differently or located elsewhere
MODEL_DIR = './fsmt_model_saved' 
MODEL_WEIGHTS = os.path.join(MODEL_DIR, 'fsmt_bert_model.bin')
MAX_LEN = 128
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Loading model from {MODEL_DIR}...")
print(f"Using device: {DEVICE}")

Loading model from ./fsmt_model_saved...
Using device: cuda


In [8]:
# Load the tokenizer from the saved directory
try:
    tokenizer = BertTokenizer.from_pretrained(MODEL_DIR)
except OSError:
    print(f"Error: Could not find tokenizer files in {MODEL_DIR}")
    print("Ensure 'vocab.txt', 'special_tokens_map.json', etc. are in the folder.")
    exit()

# Initialize the model structure
model = FSMTRegressor(n_classes=4)

# Load the trained weights
try:
    # map_location ensures it loads on CPU if you don't have a local GPU
    model.load_state_dict(torch.load(MODEL_WEIGHTS, map_location=DEVICE))
    model.to(DEVICE)
    model.eval() # Set to evaluation mode (turns off dropout)
    print("Model loaded successfully!")
except FileNotFoundError:
    print(f"Error: Could not find model weights at {MODEL_WEIGHTS}")
    exit()

C:\Users\apvan\AppData\Local\Temp\ipykernel_58500\489913744.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_WEIGHTS, map_location

Model loaded successfully!


In [10]:
def predict_fsmt(text):
    encoded_text = tokenizer.encode_plus(
        text,
        max_length=MAX_LEN,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoded_text['input_ids'].to(DEVICE)
    attention_mask = encoded_text['attention_mask'].to(DEVICE)

    with torch.no_grad():
        output = model(input_ids, attention_mask)
        prediction = output.cpu().numpy()[0]

    return {
        "Fear": prediction[0],
        "Stress": prediction[1],
        "Morale": prediction[2],
        "Trust": prediction[3]
    }

In [11]:
import json

# Dictionary to hold all loaded texts
texts = {}

for filename in os.listdir("../data/tests"):
    if filename.endswith(".txt"):
        with open("../data/tests/" + filename, "r", encoding="utf-8") as f:
            data = json.load(f)
            
            # Use filename without extension as key
            key_base = os.path.splitext(filename)[0]
                
            # Save all keys from the JSON file (like 'text', 'reference_text') inside a sub-dictionary
            texts[key_base] = data.copy()

# Example access
print(texts["drones_of_deception_temple"]["text"])
print(texts["cyber_attack_killeen_power_plant"]["reference_text"])

As tensions between Donovia and the Western Bloc reach a boiling point, reports of mysterious drone sightings in Temple, Texas, have left residents on edge and sparked concerns about foreign interference in domestic affairs. Central Texas City Grapples with Mysterious Sightings Amidst Growing Tensions The Donovian government has issued a statement denying any involvement, but insiders claim otherwise. The situation is further complicated by the escalating global power struggle between Donovia and its Western adversaries. Temple, Texas, a city in Central Texas with a population of over 70,000 residents, has become an unlikely focal point for international attention. The mysterious drone sightings have raised concerns about national security and the potential for foreign interference in domestic affairs. In recent weeks, numerous reports have surfaced from Temple residents claiming to have seen unidentified drones hovering above their homes or neighborhoods. The incidents have been chara

In [ ]:
str = 'Surprise Morning Drone Display Sparks Concerns Among Aviation Enthusiasts and General Public KILLEEN, TX - Multiple drones performing an unannounced aerial display near the Killeen Regional Airport have raised public anxiety among travelers. The incident highlights ongoing debates about drone safety regulations in the United States. Caption: KILLEEN, TX - Multiple drones performing an unannounced aerial display near the Killeen Regional Airport have raised public anxiety among travelers. The incident highlights ongoing debates about drone safety regulations in the United States. According to eyewitnesses, the drones suddenly appeared in the air around 6:00 AM local time, flying in synchronized formations and displaying bright colors. Travelers at the airport were caught off guard by the sudden display of these unmanned aerial vehicles (UAVs), which have been a source of concern for aviation enthusiasts and policymakers alike. Experts point to the increasing number of drones in the skies as a major contributor to this growing concern. As more and more people take up drone flying as a hobby, there is an ever-growing risk of these devices interfering with air traffic control systems and other aircraft operations. Drones and Air Traffic Control The Federal Aviation Administration (FAA) has set strict regulations regarding the use of drones in airspace near airports. However, enforcement can be difficult, especially when drones are flying at high altitudes or speeds. According to a recent study by the FAA, there were over 1 million reported drone sightings in US airspace last year alone. This raises serious questions about the safety and effectiveness of current regulations. Drone manufacturers argue that their devices can be designed with safety features to minimize the risk of collisions or interference with other aircraft. However, these claims are often disputed by aviation experts, who point out that such risks can never be completely eliminated. Drones and Air Traffic Control The FAA has established strict guidelines for drone pilots flying near airports, including requirements for registration, training, and airspace clearance. Despite these efforts, accidents involving drones and aircraft remain a pressing concern for aviation authorities. Drone manufacturers also argue that their devices can be equipped with advanced sensors and GPS systems to enable safe separation from other aircraft. However, such technology is still in its infancy, and further research and development are needed to ensure it is reliable and effective. The Impact on Travelers For travelers like Jane Smith, who was scheduled to board a flight at the Killeen Regional Airport but missed her connection due to the drone display, the impact of such incidents can be devastating. I was already stressed out from a long journey, she said. The thought of my flight being grounded by a bunch of noisy drones was just too much. Ill never fly again. Smiths experience is not unique. Many travelers have reported similar experiences, citing anxiety and frustration at the disruption caused by these aerial displays. The Future of Drone Regulation Solutions to this problem will require a multi-faceted approach, involving both regulatory bodies and industry stakeholders. Some propose stricter regulations on drone usage near airports, while others advocate for greater public education campaigns to raise awareness about safe flying practices. Industry leaders argue that innovative technologies can help mitigate risks associated with drone flight. The use of drones in inspection and surveying applications, for example, can provide significant benefits in terms of safety and efficiency. However, these benefits must be weighed against the potential risks posed by drone operations near airports. The debate over drone regulation is complex and multifaceted, reflecting competing interests between industries such as aviation and recreation. In order to move forward, it will require continued dialogue among all parties involved. Current Challenges Several challenges stand in the way of effective drone regulation. One major issue is the lack of standardization across different types of drones, which makes it difficult for authorities to establish clear guidelines and regulations. Another challenge is the rapid pace at which drone technology is evolving, making it difficult for regulators to keep up with the latest developments and ensure that existing rules remain relevant and effective. The Role of Technology Tech companies are playing a critical role in addressing the challenges associated with drone regulation. By investing in research and development, they can help create safer and more efficient drone operations. Several tech companies are already working on solutions to improve drone safety, such as advanced sensors and GPS systems. These technologies have the potential to significantly reduce the risk of collisions and interference between drones and other aircraft. The Impact on Airports Airports are particularly vulnerable to disruptions caused by drone operations. When a drone flies too close to an airport, it can pose a significant threat to safety, whether intentionally or unintentionally. Airport authorities must take proactive steps to mitigate these risks and ensure that drones do not interfere with air traffic control systems or other aircraft operations. Conclusion In conclusion, the incident at Killeen Regional Airport highlights the need for effective drone regulation. By investing in research and development, regulatory bodies can create safer and more efficient drone operations. Industry leaders must work together to address the challenges associated with drone regulation, including standardization, technology, and airport safety. Only through continued dialogue and collaboration can we move forward towards a future where drones are used safely and responsibly'

results = predict_fsmt(str)
        
print(f"\nResults for")
for k, v in results.items():
    print(f"  {k}: {v:.4f}")


Results for
  Fear: 0.3740
  Stress: 0.5737
  Morale: 0.4539
  Trust: 0.1999
